# Week6 - Classifier Evaluation Lab

* Copy&paste your model for homework5 model
* Add grid search and train
* Compare performance
* Which one is better? Explain?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Load DataFrame
df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/classification/loan_status_data/loan_status.csv')

#Transformations
df['Loan_Status_Transf'] = df['Loan_Status'].apply(lambda x: x.replace('Y', '1').replace('N', '0'))
df['Loan_Status_Transf'] = df['Loan_Status_Transf'].apply(lambda x: int(x))
df = df.drop(['Loan_Status'], axis=1)
df.head(5)

#Selecting features
numerical_vars = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']
categorical_vars = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Credit_History', 'Property_Area']

#Train-Test Split
from sklearn.model_selection import train_test_split

features = numerical_vars + categorical_vars
X = df[features]
y = df['Loan_Status_Transf']

#Drop NaN from the loan_amount column since it won't be accepted by the Logistic Regression model function, then re-do
#The train test split with the same random state
df = df.dropna(axis=0, how='any', subset=['Loan_Amount_Term'])

#Drop the outliers from Coapplicant income
df = df.drop(df[df['CoapplicantIncome'] > 15000].index)

X = df[features]
y = df['Loan_Status_Transf']

X_training, X_test, y_training, y_test = train_test_split(X, y, test_size=0.20, random_state=120)

print(f'Training samples: {X_training.shape[0]:,}')
print(f'Test samples: {X_test.shape[0]:,}')

Training samples: 294
Test samples: 74


In [8]:
#Run a baseline model
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

num_pipeline = Pipeline([
    ('scaling', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('encoding', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

processing_pipeline = ColumnTransformer(transformers=[
    ('num pipeline', num_pipeline, numerical_vars),
    ('cat pipeline', categorical_pipeline, categorical_vars)
])

modeling_pipeline = Pipeline([
    ('data_processing', processing_pipeline),
    ('logreg', LogisticRegression(max_iter=1000))]
)

modeling_pipeline

Pipeline(steps=[('data_processing',
                 ColumnTransformer(transformers=[('num pipeline',
                                                  Pipeline(steps=[('scaling',
                                                                   StandardScaler())]),
                                                  ['ApplicantIncome',
                                                   'CoapplicantIncome',
                                                   'LoanAmount',
                                                   'Loan_Amount_Term']),
                                                 ('cat pipeline',
                                                  Pipeline(steps=[('encoding',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Gender', 'Married',
                                                   'Dependents', 'Education',
                                                   'Self_Employed',
                                                   'Credit_History',
                                                   'Property_Area'])])),
                ('logreg', LogisticRegression(max_iter=1000))])

In [9]:
#Model fitting and predictions
loanstatusbase = modeling_pipeline.fit(X_training, y_training)
base_p = loanstatusbase.predict(X_test)

#Base classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, base_p))

              precision    recall  f1-score   support

           0       0.82      0.39      0.53        23
           1       0.78      0.96      0.86        51

    accuracy                           0.78        74
   macro avg       0.80      0.68      0.69        74
weighted avg       0.79      0.78      0.76        74



In [16]:
#Run grid search
from sklearn.model_selection import GridSearchCV

param_grid = [
  {'logreg__class_weight': ['balanced'], 'logreg__C':[0.01, 0.1, 1, 10, 100]}
 ]

gcv_results = GridSearchCV(estimator=modeling_pipeline, param_grid=param_grid, scoring='precision', refit=True)
gcv_results = gcv_results.fit(X_training, y_training)

In [17]:
gcv_results.best_params_

{'logreg__C': 100, 'logreg__class_weight': 'balanced'}

In [18]:
y_testp = gcv_results.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_testp))

              precision    recall  f1-score   support

           0       0.70      0.61      0.65        23
           1       0.83      0.88      0.86        51

    accuracy                           0.80        74
   macro avg       0.77      0.75      0.75        74
weighted avg       0.79      0.80      0.79        74



The second model is slightly better than the first model. This is because we achieved an increase in precision without a big decrease in recall. The overall accuracy is slightly higher as well, 2% to be exact. Measuring precision here is important because it would be expensive to accidentally give someone a loan that they are likely to default on.